In [1]:
import pandas as pd
from scipy import stats

survey = pd.read_csv("survey.csv")
survey.head()

,Sex,Wr.Hnd,NW.Hnd,W.Hnd,Fold,Pulse,Clap,Exer,Smoke,Height,M.I,Age
0,Female,18.5,18.0,Right,R on L,92.0,Left,Some,Never,173.0,Metric,18.250
1,Male,19.5,20.5,Left,R on L,104.0,Left,None,Regul,177.8,Imperial,17.583
2,Male,18.0,13.3,Right,L on R,87.0,Neither,None,Occas,NaN,NaN,16.917
3,Male,18.8,18.9,Right,R on L,NaN,Neither,None,Never,160.0,Metric,20.333
4,Male,20.0,20.0,Right,Neither,35.0,Right,Some,Never,165.0,Metric,23.667


In [2]:
survey_tab = pd.crosstab(survey.Smoke,survey.Exer,margins = True)
observed = survey_tab.iloc[0:4,0:3]

In [3]:
#we have created a table for smokers
observed

Exer,Freq,None,Some
Smoke,,,
Heavy,7,1,3
Never,87,18,84
Occas,12,3,4
Regul,9,1,7


In [4]:
contg = stats.chi2_contingency(observed= observed)
p_value = round(contg[1],3)
print ("P-value is: ",p_value)
#if p_val < 0.05 there is no dependency else if p_val > 0.05 there is strong dependency b/w two variables

#as the output is 0.483 which is greater than 0.05 the dependency is not there b/w the variables smoker and exercise 

P-value is:  0.483


# ANOVA
Example: A fertilizer company developed three new types of universal
fertilizers after research that can be utilized to grow any type of crop. In
order to find out whether all three have a similar crop yield, they randomly
chose six crop types in the study. In accordance with the randomized block
design, each crop type will be tested with all three types of fertilizer
separately. The following table represents the yield in g/m2
. At the 0.05 level
of significance, test whether the mean yields for the three new types of
fertilizers are all equal:

In [6]:
'''
Fertilizer 1 Fertilizer 2 Fertilizer 3
62              54            48
62              56            62
90              58            92
42              36            96
84              72            92
64              34            80'''

fetilizers = pd.read_csv("fetilizers.csv")
one_way_anova = stats.f_oneway(fetilizers["fertilizer1"],fetilizers["fertilizer2"], fetilizers["fertilizer3"])
print ("Statistic :", round(one_way_anova[0],2),", p-value:",round(one_way_anova[1],3))

Statistic : 3.66 , p-value: 0.051


# Result: The p-value did come as less than 0.05, hence we can reject the null hypothesis that the mean crop yields of the fertilizers are equal. Fertilizers make a significant difference to crops.

# dividing the data for training and testing

In [11]:
from sklearn.model_selection import train_test_split
cars = pd.read_csv("mtcars.csv")
cars.head()

,Unnamed: 0,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [12]:
train_data,test_data = train_test_split(cars,train_size=0.7,random_state = 42)

In [27]:
import numpy as np
x = np.array(train_data["hp"])
y = np.array(train_data["mpg"])
X = x.reshape(22,1)
y = y.reshape(22,1)

In [18]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  32 non-null     object 
 1   mpg         32 non-null     float64
 2   cyl         32 non-null     int64  
 3   disp        32 non-null     float64
 4   hp          32 non-null     int64  
 5   drat        32 non-null     float64
 6   wt          32 non-null     float64
 7   qsec        32 non-null     float64
 8   vs          32 non-null     int64  
 9   am          32 non-null     int64  
 10  gear        32 non-null     int64  
 11  carb        32 non-null     int64  
dtypes: float64(5), int64(6), object(1)
memory usage: 3.1+ KB


In [21]:
print(len(train_data["hp"])) 
print(len(train_data["mpg"])) 

22
22


In [23]:
cars.shape

(32, 12)

In [25]:
x.shape

(22,)

In [28]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept = True)
model.fit(X,y)
print ("Linear Regression Results" )
print ("Intercept",model.intercept_[0] ,"Coefficient", model.coef_[0])

Linear Regression Results
Intercept 30.854953531578644 Coefficient [-0.07616011]


In [ ]:
#applying gradient descent for this model with out using any inbuilt methods
def gradient_descent(x, y,learn_rate,conv_threshold,batch_size, max_iter):
    converged = False
    iter = 0
    m = batch_size 
    t0 = np.random.random(x.shape[1])
    t1 = np.random.random(x.shape[1])
    while not converged:
        grad0 = 1.0/m * sum([(t0 + t1*x[i] - y[i]) for i in range(m)])
        grad1 = 1.0/m * sum([(t0 + t1*x[i] - y[i])*x[i] for i in range(m)])
        temp0 = t0 - learn_rate * grad0
        temp1 = t1 - learn_rate * grad1
        t0 = temp0
        t1 = temp1
        MSE = 0
        MSE_New = (sum( [ (t0 + t1*x[i] - y[i])**2 for i in range(m)]) / m)
        if abs(MSE - MSE_New ) <= conv_threshold:
            print('Converged, iterations: ', iter)
            converged = True
            MSE = MSE_New
            iter += 1
        if iter == max_iter:
            print('Max interactions reached')
            converged = True
            return t0,t1
if __name__ == '__main__':
    Inter, Coeff = gradient_descent(x = X,y = y,learn_rate=0.00003 ,conv_threshold = 1e-8, batch_size=22,max_iter=1500000)
    print ('Gradient Descent Results')
    print (('Intercept = %s Coefficient = %s') %(Inter, Coeff))